In [ ]:
# Uncomment and run if installation does not exist in environment
#%pip install numpy
#%pip install pandas
#%pip install nibabel
#%pip install seaborn
#%pip install matplotlib
#%pip install nilearn
#%pip install pyrelimri

In [ ]:
import sys
import os
import argparse
import warnings
import numpy as np
import pandas as pd
import nibabel as nib
import seaborn as sns
import matplotlib.pyplot as plt
from nilearn import image, masking, plotting
from glob import glob
from pyrelimri import similarity
from itertools import product
from scipy.ndimage import binary_erosion
warnings.filterwarnings("ignore")
plt.rcParams["font.family"] = 'Times New Roman'

In [ ]:
plt.rcParams["font.family"]

In [ ]:
def extract_median(img_list, mask_img):
    data_list = []
    mask_name = os.path.basename(mask_img)
    mask = mask_name.split('.')[0]
    for img in img_list:
        base_name = os.path.basename(img)
        con, mni, motion, model, fwhm = base_name.split('_')[2:7]
    
        # get median value
        nifti_img = image.load_img(img)
        masked_data = masking.apply_mask(nifti_img, mask_img)
        median_est = np.round(np.median(masked_data), 3)
        
        data_list.append({
            'mask': mask,
            'con': con,
            'motion': motion,
            'model': model,
            'fwhm': fwhm,
            'median_est': median_est
        })
    return pd.DataFrame(data_list)

def calculate_average(img, mask):
    masked_data = masking.apply_mask(image.load_img(img), mask)
    return np.round(np.mean(masked_data), 3)
    
def roi_average(img_list, left_mask, right_mask):
    data_list = []
    for img in img_list:
        base_name = os.path.basename(img)
        con, mni, motion, model, fwhm = base_name.split('_')[2:7]
    
        # Calculate average values for left and right masks
        avg_left = calculate_average(img, left_mask)
        avg_right = calculate_average(img, right_mask)
        
        data_list.append({
            'con': con,
            'motion': motion,
            'model': model,
            'fwhm': fwhm,
            'avg_left': avg_left,
            'avg_right': avg_right
        })
    
    return pd.DataFrame(data_list)

def extract_median_rand(inp_dir,model_name, ses, task, subpra_mask, sub_mask, seed_list, n_range):
    data_list = []

    for seed_n in seed_list:
            for n in n_range:
                # set paths
                icc_path = f'{inp_dir}/seed-{seed_n}_subs-{n}_task-{task}_type-run_{model_name}_stat-est.nii.gz'
                msbtwn_path = f'{inp_dir}/seed-{seed_n}_subs-{n}_task-{task}_type-run_{model_name}_stat-msbtwn.nii.gz'
                mswthn_path = f'{inp_dir}/seed-{seed_n}_subs-{n}_task-{task}_type-run_{model_name}_stat-mswthn.nii.gz'
                # load images
                icc_img = image.load_img(icc_path)
                msbtwn_img = image.load_img(msbtwn_path)
                mswthn_img = image.load_img(mswthn_path)
                #mask images
                icc_supra = masking.apply_mask(icc_img, subpra_mask)
                icc_sub = masking.apply_mask(icc_img, sub_mask)
                msbtwn_supra = masking.apply_mask(msbtwn_img, subpra_mask)
                msbtwn_sub = masking.apply_mask(msbtwn_img, sub_mask)
                mswthn_supra = masking.apply_mask(mswthn_img, subpra_mask)
                mswthn_sub = masking.apply_mask(mswthn_img, sub_mask)
                # extract median
                icc_supra_med = np.round(np.median(icc_supra), 3)
                icc_sub_med = np.round(np.median(icc_sub), 3)
                msbtwn_supra_med = np.round(np.median(msbtwn_supra), 3)
                msbtwn_sub_med = np.round(np.median(msbtwn_sub), 3) 
                mswthn_supra_med = np.round(np.median(mswthn_supra), 3)
                mswthn_sub_med = np.round(np.median(mswthn_sub), 3) 
                
                data_list.append({
                    'seed': seed_n,
                    'subsample_n': n,
                    'icc_supra': icc_supra_med,
                    'icc_sub': icc_sub_med,
                    'msbtwn_supra': msbtwn_supra_med,
                    'msbtwn_sub': msbtwn_sub_med,
                    'mswthn_supra': mswthn_supra_med,
                    'mswthn_sub': mswthn_sub_med
                })
    return pd.DataFrame(data_list)

def file_exists(file_path):
    return os.path.exists(file_path)

# if file doesnt exist, extract media and create file, otherwise load file 
def process_dataset(imgs, mask, out_file):
    if not file_exists(out_file):
        print(f'file {out_file} doesnt exist, creating')
        df = extract_median(imgs, mask)
        df.to_csv(out_file, sep='\t', index=False)
    else:
        print(f'Loading existing {out_file}')
        df = pd.read_csv(out_file, sep='\t')
    return df

def process_dataset_roi(imgs, left_mask, right_mask, out_file):
    if not file_exists(out_file):
        print(f'file {out_file} doesnt exist, creating')
        df = roi_average(imgs, left_mask=left_mask, right_mask=right_mask)
        df.to_csv(out_file, sep='\t', index=False)
    else:
        print(f'Loading existing {out_file}')
        df = pd.read_csv(out_file, sep='\t')
    return df

def mean_sd_img(list_imgs):
    # get data and stack it
    img_list = [image.load_img(img_path) for img_path in list_imgs]
    img_data = np.stack([img.get_fdata() for img in img_list], axis=-1)
    # calculate the mean + sd image
    mean_img_data = np.mean(img_data, axis=-1)
    sd_img_data = np.std(img_data, axis=-1)
    # convert back into space of icc imgs
    icc_mean_img = image.new_img_like(img_list[0], mean_img_data)
    icc_sd_img = image.new_img_like(img_list[0], sd_img_data)
    # return mean + sd img
    return icc_mean_img,icc_sd_img

def plot_icc_maps(maps, names, coordinates, vmax):
    num_maps = len(maps)
    num_rows = num_maps // 2 + num_maps % 2  # Calculate the number of rows based on the number of maps
    fig, axes = plt.subplots(num_rows, 2, figsize=(15, 12))

    for i, map_type in enumerate(maps):
        labels = names[i].split('_')
        est_lab = labels[1]
        #thresh_lab = labels[2]

        row = i // 2
        col = i % 2
        
        if num_rows == 1:  # If only one row, axes is not 2D
            plotting.plot_stat_map(map_type, display_mode='tiled', title=f'{est_lab}',
                                   cut_coords=coordinates, vmax=vmax, axes=axes[col])
        else:
            plotting.plot_stat_map(map_type, display_mode='tiled', title=f'{est_lab}',
                                   cut_coords=coordinates, vmax=vmax, axes=axes[row, col])


    plt.tight_layout()
    plt.show()


def plot_top_map(img, img_index, coordinates, vmax):
    plotting.plot_stat_map(img[img_index], display_mode='tiled',
                           cut_coords=coordinates, vmax=vmax)
    
    plt.tight_layout()
    plt.show()
    
def plot_dist_imgs(imgs, img_index, mask_supra, mask_sub, 
                   bins=500, ylim=(0, 1), xlim=(-5, 50), dec_round=1, xlab=''):
    masked_supra_data = masking.apply_mask(imgs[img_index], mask_supra)
    masked_sub_data = masking.apply_mask(imgs[img_index], mask_sub)

    # median for plots
    med_supra = np.median(masked_supra_data)
    med_sub = np.median(masked_sub_data)

    # Plot the distribution
    plt.hist(np.round(masked_supra_data, decimals=dec_round), bins=bins, edgecolor='black', color='black',
             alpha=1, label='supra-thresh', density=True)  # supra-threshold mask
    plt.hist(np.round(masked_sub_data, decimals=dec_round), bins=bins, edgecolor='black', color='yellow',
             alpha=0.6, label='sub-thresh', density=True)  # sub-threshold mask

    plt.axvline(med_supra, color='red', linestyle='dashed', linewidth=1, label='Median supra-thresh')
    plt.axvline(med_sub, color='blue', linestyle='dashed', linewidth=1, label='Median sub-thresh')
    plt.ylim(ylim[0], ylim[1])
    plt.xlim(xlim[0], xlim[1])

    plt.xlabel(f'{xlab}',fontname='Times New Roman')
    plt.ylabel('Density',fontname='Times New Roman')

    plt.legend()
    plt.show()

def dist_cohensd(min_img, max_img, mask,  bins=500, ylim=(0, 1), xlim=(-1, 1), dec_round=1, xlab=''):
    min_data = masking.apply_mask(min_img, mask)
    max_data = masking.apply_mask(max_img, mask)

    # mean + sd for cohens d est
    mean_min = np.mean(min_data)
    mean_max = np.mean(max_data)
    sd_min = np.std(min_data, ddof=1)
    sd_max = np.std(max_data, ddof=1)

    cohensd = (mean_max - mean_min) / np.sqrt((sd_min**2+sd_max**2)/2)
    
    # Plot the distribution
    plt.hist(np.round(min_data, decimals=dec_round), bins=bins,
             alpha=.5, label='Min. Image', density=True) 
    plt.hist(np.round(max_data, decimals=dec_round), bins=bins,
             alpha=.5, label='Max. Image', density=True)  
    plt.ylim(ylim[0], ylim[1])
    plt.xlim(xlim[0], xlim[1])

    plt.xlabel(f'{xlab}', fontname='Times New Roman')
    plt.ylabel('Density',fontname='Times New Roman')
    plt.title(f"Cohen's D Between Img1 and Img2: {round(cohensd,2)}",fontname='Times New Roman')

    plt.legend()
    plt.show()

# Set Study/File info

In [ ]:
sample='abcd' # sample, AHRB, MLS or abcd
calc_type='run' # session or run 
ses='baselineYear1Arm1' # abcd opt: baselineYear1Arm1/2YearFollowUpYArm1, MLS/AHRB opts: 1/2

In [ ]:
# Set paths
if sample == 'abcd':
    task='MID'
    data_path=f'/scratch.global/mdemiden/analyses_reliability'
    out_dir= data_path
    icc_est=os.path.join(out_dir, "icc_mods", calc_type)
    group_out=os.path.join(out_dir, "group_lowmotion")
    file_out=os.path.join(out_dir,"file_output")
    analysis_path='/home/feczk001/mdemiden/analyses/Multiverse_Reliability/Stage2_Code/cluster_jobs'
    subj_list=os.path.join(analysis_path, "subj_ids", "mfd_incl.tsv")
    seed_list=os.path.join(analysis_path, "subj_ids", "rand_seeds.tsv")
    icc_resamp=os.path.join(out_dir, "subsample", "icc_subsampl",f'ses-{ses}')
    icc_resample_file=os.path.join(file_out,f'sample-{sample}_ses-baselineYear1Arm1_type-bootsrap_stats-icc.tsv')
    # load sub and supra masks
    mask_supra = os.path.join(analysis_path, "..", "brain_mask", "MNI152_wilson-supra.nii.gz")
    mask_sub = os.path.join(analysis_path, "..", "brain_mask", "MNI152_wilson-sub.nii.gz")
    left_nacc = os.path.join(analysis_path, "..", "brain_mask", "Left_NAcc.nii.gz")
    right_nacc = os.path.join(analysis_path, "..", "brain_mask", "Right_NAcc.nii.gz")
elif sample in ['AHRB','MLS']:
    data_path=os.path.join('/oak/stanford/groups/russpold/data',sample)
    out_dir=os.path.join(data_path,"derivatives","analyses","proj_reliability")
    icc_est=os.path.join(out_dir,"icc_mods",calc_type)
    group_out=os.path.join(out_dir,"group")
    analysis_path=os.path.join(data_path,"code","proj_reliability","Multiverse_Reliability","Stage2_Code","cluster_jobs")
    file_out=os.path.join(out_dir,"file_output")
    # load sub and supra masks
    mask_supra = os.path.join(analysis_path, "..", "brain_mask", "MNI152_wilson-supra.nii.gz")
    mask_sub = os.path.join(analysis_path, "..", "brain_mask", "MNI152_wilson-sub.nii.gz")
    left_nacc = os.path.join(analysis_path, "..", "brain_mask", "Left_NAcc.nii.gz")
    right_nacc = os.path.join(analysis_path, "..", "brain_mask", "Right_NAcc.nii.gz")
    if sample == 'MLS':
        task='reward'
    else:
        task='mid'
else:
    print('Incorrect sample provided. Options: abcd, AHRB, MLS')
    

In [ ]:
# load ICC images
# icc 
icc_imgs = sorted(glob(os.path.join(icc_est,"*stat-est.nii.gz")))
# msbs 
bs_imgs = sorted(glob(os.path.join(icc_est,"*stat-msbtwn.nii.gz")))
# msws
ws_imgs = sorted(glob(os.path.join(icc_est,"*stat-mswthn.nii.gz")))

print("Acquired lists:",
      len(icc_imgs),len(bs_imgs),len(ws_imgs))

In [ ]:
# icc
out_icc_supra = os.path.join(file_out,f'sample-{sample}_type-{calc_type}_stats-est_mask-wilson-supra.tsv')
out_icc_sub = os.path.join(file_out,f'sample-{sample}_type-{calc_type}_stats-est_mask-wilson-sub.tsv')
out_icc_nacc = os.path.join(file_out,f'sample-{sample}_type-{calc_type}_stats-est_mask-nacc.tsv')
# msbs
out_bs_supra = os.path.join(file_out,f'sample-{sample}_type-{calc_type}_stats-bs_mask-wilson-supra.tsv')
out_bs_sub = os.path.join(file_out,f'sample-{sample}_type-{calc_type}_stats-bs_mask-wilson-sub.tsv')
out_bs_nacc = os.path.join(file_out,f'sample-{sample}_type-{calc_type}_stats-bs_mask-nacc.tsv')

# msw
out_ws_supra = os.path.join(file_out,f'sample-{sample}_type-{calc_type}_stats-ws_mask-wilson-supra.tsv')
out_ws_sub = os.path.join(file_out,f'sample-{sample}_type-{calc_type}_stats-ws_mask-wilson-sub.tsv')
out_ws_nacc = os.path.join(file_out,f'sample-{sample}_type-{calc_type}_stats-ws_mask-nacc.tsv')

# similarity
out_similarity= os.path.join(file_out,f'sample-{sample}_type-{calc_type}_stats-similarity.tsv')

## Calc concatenated df for model efficiences (motion, parameteization)

In [ ]:
# Model permutations
if sample in ['abcd', 'AHRB']:
    voxel = 2.4
    opts = np.array([1.5,  2, 2.5, 3, 3.5])
    fwhm_opt = list(np.round(voxel * opts, 1))
elif sample in 'MLS':
    voxel = 4
    inh_smooth_weight = .50
    opts = np.array([1.5, 2, 2.5, 3, 3.5])*inh_smooth_weight
    fwhm_opt = list(np.round(voxel * opts, 1))

# only including 4; opt 5 is opt3 + subj mFD < .9 & opt6 is opt4 + subj mFD < .9. 
motion_opt = ["opt1", "opt2", "opt3", "opt4"]
modtype_opt = ["CueMod", "AntMod", "FixMod"]

permutation_list = list(product(fwhm_opt, motion_opt, modtype_opt))

In [ ]:
if sample == 'abcd':
    ses="baselineYear1Arm1"
    first_path = os.path.join(out_dir,"firstlvl",f'ses-{ses}')
    with open(subj_list, 'r') as file:
        directories = [line.strip() for line in file.readlines()]
    print(f'{sample} subjects: {len(directories)}')
else:
    first_path = os.path.join(out_dir,"firstlvl",f'ses-1')
    directories = [d for d in os.listdir(first_path) if os.path.isdir(os.path.join(first_path, d))]
    print(f'{sample} subjects: {len(directories)}')

In [ ]:
run = ['1','2']
df_eff = []
n = 0
if sample == 'abcd':
    for subject in directories:
        # check and skip subjs w/o acompcor: 
        acomp_patt = os.path.join(first_path},subject,f'{subject}_ses-{ses}_task-{task}_*mot-opt3*beta.nii.gz')
        n_acompcor = glob(acomp_patt)
        if len(n_acompcor) == 0:
            n=n+1
            print(f'{n}. {subject} bad acompcor, skipping')
        else:
            df_r1 = pd.read_csv(os.path.join(first_path,subject,f'{subject}_ses-{ses}_task-{task}_run-0{run[0]}_efficiency.tsv'), sep=',')
            df_r2 = pd.read_csv(os.path.join(first_path,subject,f'{subject}_ses-{ses}_task-{task}_run-0{run[1]}_efficiency.tsv'), sep=',')
            for i, column_name in enumerate(['fwhm', 'motion', 'model']):
                df_r1[column_name] = [item[i] for item in permutation_list]
                df_r2[column_name] = [item[i] for item in permutation_list]
                
            df_r1.drop_duplicates(subset=['model', 'motion'], inplace=True)
            df_r2.drop_duplicates(subset=['model', 'motion'], inplace=True)
            df_concatenated = pd.concat([df_r1, df_r2])
            df_average = df_concatenated.groupby(['model', 'motion']).mean().reset_index()
            df_average = df_average.drop(['run', 'fwhm'], axis=1)
            df_average['subject'] = subject
            
            df_eff.append(df_average)
        
    df_eff = pd.concat(df_eff, ignore_index=True)
else:
    for subject in directories:
        df_r1 = pd.read_csv(os.path.join(first_path,subject,f'{subject}_ses-1_task-{task}_run-0{run[0]}_efficiency.tsv'), sep=',')
        df_r2 = pd.read_csv(os.path.join(first_path,subject,f'{subject}_ses-1_task-{task}_run-0{run[1]}_efficiency.tsv'), sep=',')
        for i, column_name in enumerate(['fwhm', 'motion', 'model']):
            df_r1[column_name] = [item[i] for item in permutation_list]
            df_r2[column_name] = [item[i] for item in permutation_list]
            
        df_r1.drop_duplicates(subset=['model', 'motion'], inplace=True)
        df_r2.drop_duplicates(subset=['model', 'motion'], inplace=True)
        df_concatenated = pd.concat([df_r1, df_r2])
        df_average = df_concatenated.groupby(['model', 'motion']).mean().reset_index()
        df_average = df_average.drop(['run', 'fwhm'], axis=1)
        df_average['subject'] = subject
    
    df_eff.append(df_average)
    
    df_eff = pd.concat(df_eff, ignore_index=True)

In [ ]:
columns_to_melt = ["Lgain-Neut", "Sgain-Neut", "Lgain-Base", "Sgain-Base"]

# convert wide to long format
df_eff_lg = pd.melt(df_eff, id_vars=['subject', 'model', 'motion'], 
                      value_vars=columns_to_melt, var_name='con', value_name='eff_est')

# efficiency 
if not os.path.exists(file_out):
    os.makedirs(file_out)
    
out_efficiency= os.path.join(file_out,f'sample-{sample}_ses-{ses}_stats-efficiency.tsv')
df_eff_lg.to_csv(out_efficiency, sep='\t', index=False)

# Means and Stand Dev of Estimates

For Aim 1 (ICC estimate) and Aim 2 (Mean Square Between Subject and Within Subject Variance), stated would include:

"First, voxel-wise average and standard deviation from the resulting ICCs for the 360 model permutations are reported in two 3D volumes."

Selected coordinated (L) Ventral Striatum based on Neurosynth search term "ventral striatum" threshold > 20

## ICC est

In [ ]:
icc_mean, icc_sd = mean_sd_img(icc_imgs)
#icc_mean_sub, icc_sd_sub = mean_sd_img(icc_sub_imgs)

In [ ]:
icc_maps = [icc_mean, icc_sd]
icc_names = ['icc_mean', 'icc_sd']

plot_icc_maps(maps=icc_maps, names=icc_names, 
              coordinates=(-10, 8, -10),vmax=1)

## MSBS est

In [ ]:
bs_mean, bs_sd = mean_sd_img(bs_imgs)

In [ ]:
bs_maps = [bs_mean, bs_sd]
bs_names = ['bs_mean', 'bs_sd']

plot_icc_maps(maps=bs_maps, names=bs_names, 
              coordinates=(-10, 8, -10),vmax=3)

## MSWS est

In [ ]:
ws_mean, ws_sd = mean_sd_img(ws_imgs)

In [ ]:
ws_maps = [ws_mean, ws_sd]
ws_names = ['ws_mean', 'ws_sd']

plot_icc_maps(maps=ws_maps, names=ws_names, 
              coordinates=(-10, 8, -10),vmax=3)

# Loading files, Extracting median ICC and providing distribution

For Aim 1 and Aim 2 state would include:
"Second, the range and distribution of median ICCs across each study (three) and analytic decision category (four) are plotted across suprathreshold task-positive and subthreshold ICCs using Rainclouds (Allen et al., 2019) and the median and standard deviation is reported in a table."

While included here to rank and contain some information, plots will be created in R using raincloud features that are more powerful

In [ ]:
# icc datasets
df_icc_supra = process_dataset(icc_imgs, mask_supra, out_icc_supra)
df_icc_sub = process_dataset(icc_imgs, mask_sub, out_icc_sub)
df_icc_nacc = process_dataset_roi(icc_imgs, left_mask=left_nacc, right_mask=right_nacc, out_file=out_icc_nacc)
# msbs datasets
df_bs_supra = process_dataset(bs_imgs, mask_supra, out_bs_supra)
df_bs_sub = process_dataset(bs_imgs, mask_sub, out_bs_sub)
df_bs_nacc = process_dataset_roi(bs_imgs, left_mask=left_nacc, right_mask=right_nacc, out_file=out_bs_nacc)

# msws datasets
df_ws_supra = process_dataset(ws_imgs, mask_supra, out_ws_supra)
df_ws_sub = process_dataset(ws_imgs, mask_sub, out_ws_sub)
df_ws_nacc = process_dataset_roi(ws_imgs, left_mask=left_nacc, right_mask=right_nacc, out_file=out_ws_nacc)

### sort ICC by median est + img of highest

In [ ]:
df_icc_supra.sort_values(by='median_est', ascending=False)

In [ ]:
df_icc_sub.sort_values(by='median_est', ascending=False)

In [ ]:
icc_ind = 174
plot_top_map(img=icc_imgs, img_index=icc_ind, coordinates=(-10, 8, -10), vmax=1)

In [ ]:
model_name = df_icc_supra.loc[icc_ind, ['con','motion', 'model', 'fwhm']].values
xlab = f'ICC Type:\n {", ".join(model_name)}'

plot_dist_imgs(imgs=icc_imgs, img_index=icc_ind,
               mask_supra=mask_supra, mask_sub=mask_sub,
               bins=100, ylim=(0,5),xlim=(-1,1),dec_round=3,
               xlab=xlab)

In [ ]:
icc_ind = 24
plot_top_map(img=icc_imgs, img_index=icc_ind, coordinates=(-10, 8, -10), vmax=1)

#### select lowest/highest ICC from specr

In [ ]:
# match to top session test-retest model
conditions = (
    (df_icc_supra["mask"] == "MNI152_wilson-supra") &
    (df_icc_supra["con"] == "contrast-Sgain-Neut") &
    (df_icc_supra["motion"] == "mot-opt3") &
    (df_icc_supra["model"] == "mod-FixMod") &
    (df_icc_supra["fwhm"] == "fwhm-3.6")
)

df_icc_supra[conditions].sort_values(by='median_est', ascending=False)

In [ ]:
icc_lowest = 100
plot_top_map(img=icc_imgs, img_index=icc_lowest, coordinates=(-10, 8, -10), vmax=1)

In [ ]:
model_name = df_icc_supra.loc[icc_lowest, ['con', 'motion', 'model', 'fwhm']].values
xlab = f'Alt ICC Type:\n {", ".join(model_name)}'

plot_dist_imgs(imgs=icc_imgs, img_index=icc_lowest,
               mask_supra=mask_supra, mask_sub=mask_sub,
               bins=100, ylim=(0,5),xlim=(-1,1),dec_round=3,
               xlab=xlab)

In [ ]:
# match to top session test-retest model
conditions = (
    (df_icc_supra["mask"] == "MNI152_wilson-supra") &
    (df_icc_supra["con"] == "contrast-Sgain-Base") &
    (df_icc_supra["motion"] == "mot-opt2") &
    (df_icc_supra["model"] == "mod-CueMod") &
    (df_icc_supra["fwhm"] == "fwhm-8.4")
)

df_icc_supra[conditions].sort_values(by='median_est', ascending=False)

In [ ]:
icc_highest = 204
plot_top_map(img=icc_imgs, img_index=icc_highest, coordinates=(-10, 8, -10), vmax=1)

In [ ]:
model_name = df_icc_supra.loc[icc_highest, ['con', 'motion', 'model', 'fwhm']].values
xlab = f'Alt ICC Type:\n {", ".join(model_name)}'

plot_dist_imgs(imgs=icc_imgs, img_index=icc_highest,
               mask_supra=mask_supra, mask_sub=mask_sub,
               bins=100, ylim=(0,5),xlim=(-1,1),dec_round=3,
               xlab=xlab)

### sort MSBS by median est + img of highest

In [ ]:
df_bs_supra.sort_values(by='median_est', ascending=False)

In [ ]:
df_bs_sub.sort_values(by='median_est', ascending=False)

In [ ]:
bs_ind = 185
plot_top_map(img=bs_imgs, img_index=bs_ind, coordinates=(-10, 8, -10), vmax=5)

In [ ]:
model_name = df_bs_supra.loc[bs_ind, ['con', 'motion','model', 'fwhm']].values
xlab = f'MSBS Model Type:\n {", ".join(model_name)}'

plot_dist_imgs(imgs=bs_imgs, img_index=bs_ind,
               mask_supra=mask_supra, mask_sub=mask_sub,
               bins=500,ylim=(0,1),xlim=(-5,50),
               xlab=xlab)

### sort MSWS by median est + img of highest

In [ ]:
df_ws_supra.sort_values(by='median_est', ascending=False)

In [ ]:
df_ws_sub.sort_values(by='median_est', ascending=False)

In [ ]:
ws_ind = 185
plot_top_map(img=ws_imgs, img_index=ws_ind, coordinates=(-10, 8, -10), vmax=5)

In [ ]:
model_name = df_ws_supra.loc[ws_ind, ['con', 'motion','model', 'fwhm']].values
xlab = f'MSWS Type:\n {", ".join(model_name)}'

plot_dist_imgs(imgs=ws_imgs, img_index=ws_ind,
               mask_supra=mask_supra, mask_sub=mask_sub,
               bins=500,ylim=(0,5),xlim=(-5,50),
               xlab=xlab)

# Group Maps for supplemental

In [ ]:
os.path.join(group_out,f'ses-{ses}','run'),
glob(os.path.join(group_maps,f'subs-*_ses-{ses}_task-{task}_type-{run}_{cue}_stat-cohensd.nii.gz'))[0]

In [ ]:
cue='contrast-Lgain-Base_mask-mni152_mot-opt2_mod-CueMod_fwhm-8.4'
fix='contrast-Lgain-Base_mask-mni152_mot-opt2_mod-FixMod_fwhm-8.4'
ant='contrast-Lgain-Base_mask-mni152_mot-opt2_mod-AntMod_fwhm-8.4'
run='run02'
group_maps=os.path.join(group_out,f'ses-{ses}','run')
cue_map=glob(os.path.join(group_maps,f'subs-*_ses-{ses}_task-{task}_type-{run}_{cue}_stat-cohensd.nii.gz'))[0]
fix_map=glob(os.path.join(group_maps,f'subs-*_ses-{ses}_task-{task}_type-{run}_{fix}_stat-cohensd.nii.gz'))[0]
ant_map=glob(os.path.join(group_maps,f'subs-*_ses-{ses}_task-{task}_type-{run}_{ant}_stat-cohensd.nii.gz'))[0]
grp_maps = [cue_map,fix_map,ant_map]
grp_labs = ["model_cue-LgBase","model_fix-LgBase","model_ant-LgBase"]
plot_icc_maps(maps=grp_maps, names=grp_labs, 
              coordinates=(-10, 8, -10),vmax=1)

In [ ]:
cue='contrast-Lgain-Neut_mask-mni152_mot-opt2_mod-CueMod_fwhm-8.4'
fix='contrast-Lgain-Neut_mask-mni152_mot-opt2_mod-FixMod_fwhm-8.4'
ant='contrast-Lgain-Neut_mask-mni152_mot-opt2_mod-AntMod_fwhm-8.4'
run='run02'
group_maps=os.path.join(group_out,f'ses-{ses}','run')
cue_map=glob(os.path.join(group_maps,f'subs-*_ses-{ses}_task-{task}_type-{run}_{cue}_stat-cohensd.nii.gz'))[0]
fix_map=glob(os.path.join(group_maps,f'subs-*_ses-{ses}_task-{task}_type-{run}_{fix}_stat-cohensd.nii.gz'))[0]
ant_map=glob(os.path.join(group_maps,f'subs-*_ses-{ses}_task-{task}_type-{run}_{ant}_stat-cohensd.nii.gz'))[0]
grp_maps = [cue_map,fix_map,ant_map]
grp_labs = ["model_cue-LgNeut","model_fix-LgNeut","model_ant-LgNeut"]
plot_icc_maps(maps=grp_maps, names=grp_labs, 
              coordinates=(-10, 8, -10),vmax=1)

# Residual Variance ~ ICC / MSWS / Cohens D

In [ ]:
first_path = f'{out_dir}/firstlvl'
ses1_out = f'{group_out}/abcd_run'
ses2_out = f'{group_out}/abcd_run'
ses_out = f'{group_out}/ses_residvar'
con_opts = ['Lgain-Neut','Lgain-Base','Sgain-Neut','Sgain-Base']
model_opts = ['CueMod','AntMod','FixMod']
task = 'mid'
fwhm = 'fwhm-6.0'
n = 

if not os.path.exists(ses_out):
    os.makedirs(ses_out)

In [ ]:
# dont re-run if already ran
for con in con_opts:
    for mod in model_opts:    
        resid_ses1_imgs = glob(f'{first_path}/ses-1/**/*_ses-1_task-{task}_run-0*_contrast-{con}_mask-mni152_mot-opt1_mod-{mod}_{fwhm}_stat-residvar.nii.gz')    
        resid_ses2_imgs = glob(f'{first_path}/ses-2/**/*_ses-2_task-{task}_run-0*_contrast-{con}_mask-mni152_mot-opt1_mod-{mod}_{fwhm}_stat-residvar.nii.gz')      

        ses1_mean, ses1_sd = mean_sd_img(resid_ses1_imgs)
        ses2_mean, ses2_sd = mean_sd_img(resid_ses1_imgs)
        ses_mean, ses_sd = mean_sd_img([ses1_mean,ses2_mean])
        
        ses1_name = f'{ses1_out}/subs-{n}_ses-1_task-{task}_type-session_contrast-{con}_mask-mni152_mot-opt1_mod-{mod}_{fwhm}_stat-residvar.nii.gz'
        ses2_name = f'{ses2_out}/subs-{n}_ses-2_task-{task}_type-session_contrast-{con}_mask-mni152_mot-opt1_mod-{mod}_{fwhm}_stat-residvar.nii.gz'
        ses_name = f'{ses_out}/subs-{n}_task-{task}_contrast-{con}_mask-mni152_mot-opt1_mod-{mod}_{fwhm}_stat-residvar.nii.gz'

        nib.save(ses1_mean, ses1_name)
        nib.save(ses2_mean, ses2_name)
        nib.save(ses_mean, ses_name)


In [ ]:
con = 'Lgain-Neut'
ant_mod = f'{ses_out}/subs-{n}_task-{task}_contrast-{con}_mask-mni152_mot-opt1_mod-AntMod_{fwhm}_stat-residvar.nii.gz'
fix_mod = f'{ses_out}/subs-{n}_task-{task}_contrast-{con}_mask-mni152_mot-opt1_mod-FixMod_{fwhm}_stat-residvar.nii.gz'
cue_mod = f'{ses_out}/subs-{n}_task-{task}_contrast-{con}_mask-mni152_mot-opt1_mod-CueMod_{fwhm}_stat-residvar.nii.gz'


In [ ]:
bins = 50

ant_supra = masking.apply_mask(ant_mod, mask_supra)
fix_supra = masking.apply_mask(fix_mod, mask_supra)
cue_supra = masking.apply_mask(cue_mod, mask_supra)
mod_maps = [ant_supra, fix_supra, cue_supra]
mod_names = ["AntMod","FixMod","CueMod"]
colors = ['white','yellow','blue']

fig, axes = plt.subplots(3, 1, figsize=(15, 12))

for i, map_mod in enumerate(mod_maps):
    label = mod_names[i]
    color = colors[i]
    axes[i].hist(np.round(map_mod, decimals=1), bins=bins, edgecolor='black', color=color,
                 alpha=1, label=label, density=True)
    axes[i].legend() 
    axes[i].set_ylim(0,5)
    axes[i].set_xlim(0,4)
    axes[i].set_title(f"{con}")
    axes[i].set_ylabel('Density')
plt.tight_layout()
plt.show()


In [ ]:
print(out_dir, group_out)

In [ ]:
mod_names = ["AntMod","FixMod","CueMod"]
con_names = ['Lgain-Neut','Lgain-Base','Sgain-Neut','Sgain-Base']
est_names = ['est', 'msbtwn', 'mswthn']
ses = 'baselineYear1Arm1'
first_path = f'{out_dir}/firstlvl'
ses1_out = f'{group_out}/abcd_session/baselineYear1Arm1'
ses2_out = f'{group_out}/abcd_session/2YearFollowUpArm1'


rho_list = []
for est in est_names:
    for mod in mod_names:
        for con in con_names:
            est_maps=glob(f'{out_dir}/icc_perm/run/*mask-wilson-supra_*contrast-{con}_mask-mni152_mot-opt4_mod-AntMod_{fwhm}_*stat-{est}.nii.gz')
            mod_img = f'{ses_out}/*_contrast-{con}_mask-mni152_mot-opt4_mod-{mod}_{fwhm}_stat-residvar.nii.gz'
            grp_img = f'{ses1_out}/*_task-{task}_type-session_contrast-{con}_mask-mni152_mot-opt4_mod-{mod}_{fwhm}_stat-cohensd.nii.gz'
            rho_icc = similarity.image_similarity(imgfile1=est_maps[0], imgfile2=mod_img, mask=mask_supra, similarity_type = 'spearman')
            rho_icccohens = similarity.image_similarity(imgfile1=est_maps[0], imgfile2=grp_img, mask=mask_supra, similarity_type = 'spearman')
            rho_residcohens = similarity.image_similarity(imgfile1=mod_img, imgfile2=grp_img, mask=mask_supra, similarity_type = 'spearman')
            rho_list.append({
                'Model': mod,
                'Contrast': con,
                'estimate': est,
                'icc_rho': rho_icc,
                'cohensicc_rho': rho_icccohens,
                'coehnsresid_rho': rho_residcohens
                
            })


In [ ]:
contrast = "Lgain-Base"
vmin = -1
vmax = 1 

rho_df = pd.DataFrame(rho_list)
heatmap_rho = rho_df[rho_df["Contrast"] == contrast].pivot_table(index='estimate', 
                                                                     columns=['Model'], values='icc_rho')
plt.figure(figsize=(12, 8))
sns.heatmap(heatmap_rho, annot=True, cmap='vlag', linewidths=.5, fmt=".2f", vmin=vmin, vmax=vmax)
plt.title(f'{sample}: session ICC {contrast} estimates ~ avg {contrast} residual var')
plt.show()

In [ ]:
rho_df = pd.DataFrame(rho_list)
heatmap_rho = rho_df[rho_df["Contrast"] == contrast].pivot_table(index='estimate', 
                                                                     columns=['Model'], values='cohensicc_rho')
plt.figure(figsize=(12, 8))
sns.heatmap(heatmap_rho, annot=True, cmap='vlag', linewidths=.5, fmt=".2f", vmin=vmin, vmax=vmax)
plt.title(f'{sample}: session {contrast} cohens d ~ session {contrast} ICC estimates')
plt.show()

In [ ]:
rho_df = pd.DataFrame(rho_list)
heatmap_rho = rho_df[rho_df["Contrast"] == contrast].pivot_table(index='Contrast', 
                                                                 columns=['Model'], values='coehnsresid_rho')
plt.figure(figsize=(10, 2))
sns.heatmap(heatmap_rho, annot=True, cmap='vlag', linewidths=.5, fmt=".2f", vmin=vmin, vmax=vmax)
plt.title(f'{sample}: session {contrast} cohens d ~ session {contrast} residual var')
plt.show()

In [ ]:
rho_list = []
for est in est_names:
    for mod in mod_names:
        for con in con_names:
            est_maps=glob(f'{out_dir}/icc_mods/run/wilson-supra/*contrast-{con}_mask-mni152_mot-opt1_mod-AntMod_{fwhm}_*stat-{est}.nii.gz')
            mod_img = f'{ses1_out}/subs-{n}_{ses}_task-{task}_type-session_contrast-{con}_mask-mni152_mot-opt1_mod-{mod}_{fwhm}_stat-residvar.nii.gz'
            rho_maps = similarity.image_similarity(imgfile1=est_maps[0], imgfile2=mod_img, mask=mask_supra, similarity_type = 'spearman')
            rho_list.append({
            'Model': mod,
            'Contrast': con,
            'estimate': est,
            'rho': rho_maps
        })

In [ ]:
rho_df = pd.DataFrame(rho_list)
heatmap_rho = rho_df.pivot_table(index='Contrast', columns=['estimate', 'Model'], values='rho')
plt.figure(figsize=(12, 8))
sns.heatmap(heatmap_rho, annot=True, cmap='magma', linewidths=.5, fmt=".2f")
plt.title(f'{sample}: Heatmap for run est ~ avg residvar')
plt.show()

# Group-wise similarity

## Jaccards & spearman similarity

As described in the registered report:

*"Jaccard similarity coefficient is sensitive to thresholding and sample size (Bennett & Miller, 2010), in Aim 1 an equal sample size (e.g., N ~ 60) is chosen for each study to compare how the similarity between sessions varies across studies. For all 360 pipelines, a group-level (average) activation map is estimated for each session. In the case of the Jaccard coefficient, the group maps are thresholded at p < .001. In the case of the Spearman coefficient, the group maps are masked using a suprathreshold task-positive map from NeuroVault. Then, the paths for the pipelines and sessions are called using the pairwise_similarity within the similarity.py script. The resulting coefficients report the similarity between analytic pipelines and sessions for each study. For each study, the coefficients are plotted to reflect the distribution and range of coefficients. Both Jaccards and Spearman correlation are reported separately. Like Aim 1 & Aim 2, two HLMs are used to regress the Jaccard coefficients and Spearman correlation on the [four] analytic decisions nested within study. Multiple comparisons corrections are applied using the Tukey adjustment."*

In [ ]:
contrasts = [
    'Lgain-Neut', 'Sgain-Neut',
    'Lgain-Base', 'Sgain-Base'
]

# Model permutations
if sample in ['abcd', 'AHRB']:
    voxel = 2.4
    opts = np.array([1.5,  2, 2.5, 3, 3.5])
    fwhm_opt = list(np.round(voxel * opts, 1))
elif sample in 'MLS':
    voxel = 4
    inh_smooth_weight = .50
    opts = np.array([1.5, 2, 2.5, 3, 3.5])*inh_smooth_weight
    fwhm_opt = list(np.round(voxel * opts, 1))

# only including 4; opt 5 is opt3 + subj mFD < .9 & opt6 is opt4 + subj mFD < .9. 
motion_opt = ["opt1", "opt2", "opt3", "opt4"]
modtype_opt = ["CueMod", "AntMod", "FixMod"]

permutation_list = list(product(contrasts, fwhm_opt, motion_opt, modtype_opt))

In [ ]:
threshold = .4 # mod since t -> d transform
similar_list = []
mask_lab = 'wilson-supra'
n = 0
for con, fwhm, motion, model in permutation_list:
    n = n + 1
    print(f'Running {n} out of {len(permutation_list)}')
    # jaccard/ spearman similarity between session 1 and session 2
    img1 = f'{group_out}/ses-{ses}/{calc_type}/*type-{calc_type}01_contrast-{con}_mask-mni152_mot-{motion}_mod-{model}_fwhm-{fwhm}_stat-cohensd.nii.gz'
    img2 = f'{group_out}/ses-{ses}/{calc_type}/*type-{calc_type}02_contrast-{con}_mask-mni152_mot-{motion}_mod-{model}_fwhm-{fwhm}_stat-cohensd.nii.gz'
    jaccard = similarity.image_similarity(imgfile1=img1, imgfile2=img2, thresh = threshold, similarity_type = 'jaccard')
    spearman = similarity.image_similarity(imgfile1=img1, imgfile2=img2, mask=mask_supra, similarity_type = 'spearman')
    
    # spearman rho between session ICC + sess 1 & sess 2 cohens d
    ses_icc = f'{icc_est}/{mask_lab}/*_contrast-{con}_mask-mni152_mot-{motion}_mod-{model}_fwhm-{fwhm}*stat-est.nii.gz'
    run1 = f'{group_out}/ses-{ses}/{calc_type}/*type-{calc_type}01_contrast-{con}_mask-mni152_mot-{motion}_mod-{model}_fwhm-{fwhm}_stat-cohensd.nii.gz'
    run2 = f'{group_out}/ses-{ses}/{calc_type}/*type-{calc_type}02_contrast-{con}_mask-mni152_mot-{motion}_mod-{model}_fwhm-{fwhm}_stat-cohensd.nii.gz'
    run1_spear = similarity.image_similarity(imgfile1=ses_icc, imgfile2=run1, mask=mask_supra, similarity_type = 'spearman')
    run2_spear = similarity.image_similarity(imgfile1=ses_icc, imgfile2=run2, mask=mask_supra, similarity_type = 'spearman')


    similar_list.append({
        'con': con,
        'fwhm': fwhm,
        'motion': motion,
        'model': model,
        'jaccard': jaccard,
        'spearman': spearman,
        'run1_icc_cohensd': run1_spear,
        'run2_icc_cohensd': run2_spear,
        
    })
 

In [ ]:
similarity_df = pd.DataFrame(similar_list)
similarity_df.to_csv(out_similarity, sep='\t', index=False)

## Posthc, Session Group Map ~ Session ICC

In [ ]:
spear_sim_df = pd.DataFrame(similarity_df[['run1_icc_cohensd','run2_icc_cohensd']])
spear_sim_long = pd.melt(spear_sim_df, var_name="Session", value_name="SpearmanEstimate")

# Boxplot Spearman rho between Session 1 (or 2) Group Cohen's D map ~ Session ICC est. masked suprathresh
plt.figure(figsize=(8, 6))
sns.boxplot(x="Session", y="SpearmanEstimate", data=spear_sim_long)
plt.ylabel('Estimate')
plt.title('Spearman rho: Session ICC ~ Cohens D for Run 1 and Run 2')

# Show the plot
plt.show()

# ICC, MSBS, MSWS random Sampling (ABCD Only)

In [ ]:
seed_ns = pd.read_csv(seed_list, sep='\t', header=None)
seed_ns_list = seed_ns.iloc[:, 0].tolist()
ses='baselineYear1Arm1'
n_int = 50
min_n = 25
max_n = 525
n_range = list(range(min_n, max_n+n_int, n_int))
model='contrast-Lgain-Base_mask-mni152_mot-opt2_mod-CueMod_fwhm-8.4'

In [ ]:
if not file_exists(icc_resample_file):
        print(f'file {icc_resample_file} doesnt exist, creating')
        df_boot = extract_median_rand(inp_dir=icc_resamp,model_name=model, ses=ses, task='MID', 
                                      subpra_mask=mask_supra, sub_mask=mask_sub, 
                                      seed_list=seed_ns_list, n_range=n_range)
        df_boot.to_csv(icc_resample_file, sep='\t', index=False)
else:
        print(f'Loading existing {icc_resample_file}')
        df_boot = pd.read_csv(icc_resample_file, sep='\t')


In [ ]:
n1 = 25
sort_desc= df_boot[df_boot["subsample_n"] == n1].sort_values(by='icc_supra', ascending=True)
seed_lowest = sort_desc["seed"].iloc[0]
sort_ascend= df_boot[df_boot["subsample_n"] == n1].sort_values(by='icc_supra', ascending=False)
seed_highest = sort_ascend["seed"].iloc[0]
# plot
img1_25 = f'{icc_resamp}/seed-{seed_lowest}_subs-{n1}_task-MID_type-run_{model}_stat-est.nii.gz'
img2_25 = f'{icc_resamp}/seed-{seed_highest}_subs-{n1}_task-MID_type-run_{model}_stat-est.nii.gz'
dist_cohensd(min_img=img1_25, max_img=img2_25, mask=mask_supra,  bins=200, ylim=(0, 5), xlim=(-1, 1), dec_round=4, xlab='')

In [ ]:
n2 = 225
sort_desc= df_boot[df_boot["subsample_n"] == n2].sort_values(by='icc_supra', ascending=True)
seed_lowest = sort_desc["seed"].iloc[0]
sort_ascend= df_boot[df_boot["subsample_n"] == n2].sort_values(by='icc_supra', ascending=False)
seed_highest = sort_ascend["seed"].iloc[0]
# plot
img1_225 = f'{icc_resamp}/seed-{seed_lowest}_subs-{n2}_task-MID_type-run_{model}_stat-est.nii.gz'
img2_225 = f'{icc_resamp}/seed-{seed_highest}_subs-{n2}_task-MID_type-run_{model}_stat-est.nii.gz'
dist_cohensd(min_img=img1_225, max_img=img2_225, mask=mask_supra,  bins=200, ylim=(0, 5), xlim=(-1, 1), dec_round=4, xlab='')

In [ ]:
n3 = 525
sort_desc= df_boot[df_boot["subsample_n"] == n3].sort_values(by='icc_supra', ascending=True)
seed_lowest = sort_desc["seed"].iloc[0]
sort_ascend= df_boot[df_boot["subsample_n"] == n3].sort_values(by='icc_supra', ascending=False)
seed_highest = sort_ascend["seed"].iloc[0]
# plot
img1_525 = f'{icc_resamp}/seed-{seed_lowest}_subs-{n3}_task-MID_type-run_{model}_stat-est.nii.gz'
img2_525 = f'{icc_resamp}/seed-{seed_highest}_subs-{n3}_task-MID_type-run_{model}_stat-est.nii.gz'
dist_cohensd(min_img=img1_525, max_img=img2_525, mask=mask_supra,  bins=200, ylim=(0, 5), xlim=(-1, 1), dec_round=4, xlab='')

In [ ]:
n_25_imgs = [img1_25,img2_25]
plot_top_map(img=n_25_imgs, img_index=0, coordinates=(-10, 8, -10), vmax=1),plot_top_map(img=n_25_imgs, img_index=1, coordinates=(-10, 8, -10), vmax=1)

In [ ]:
n_225_imgs = [img1_225,img2_225]
plot_top_map(img=n_225_imgs, img_index=0, coordinates=(-10, 8, -10), vmax=1),plot_top_map(img=n_225_imgs, img_index=1, coordinates=(-10, 8, -10), vmax=1)

In [ ]:
n_525_imgs = [img1_525,img2_525]
plot_top_map(img=n_525_imgs, img_index=0, coordinates=(-10, 8, -10), vmax=1),plot_top_map(img=n_525_imgs, img_index=1, coordinates=(-10, 8, -10), vmax=1)